In [ ]:
import pandas as pd

file_path = "/content/Glencore recap 11062025.xlsx"
xls = pd.ExcelFile(file_path)
sheet_data = xls.parse("Sheet1", header=None)

weekly_cfds_rows = []
collecting = False

for index, row in sheet_data.iterrows():
    first_cell = str(row[0]).strip()

    if first_cell.startswith("Week1"):
        collecting = True

    if collecting:
        if str(first_cell).startswith("Week"):
            weekly_cfds_rows.append([row[0], row[1], row[3], row[4]])
        else:
            break

weekly_cfds_df = pd.DataFrame(weekly_cfds_rows, columns=['Week', 'Date Range', 'Month', 'Value'])
weekly_cfds_df = weekly_cfds_df.reset_index(drop=True)

dfls_rows = []

for index, row in sheet_data.iterrows():
    month = row[7]
    value = row[8]

    if pd.notna(month) and pd.notna(value):
        dfls_rows.append([month, value])

dfls_df = pd.DataFrame(dfls_rows, columns=['Month', 'Value'])

spreads_rows = []

for index, row in sheet_data.iterrows():
    col3 = row[3]
    col4 = row[4]

    if pd.notna(col3) and pd.notna(col4):
        if "ICE" in str(col3) or "/" in str(col3):
            spreads_rows.append([col3, col4])

spreads_df = pd.DataFrame(spreads_rows, columns=['Spread', 'Value'])

efps_from_dfls_df = dfls_df.tail(3).reset_index(drop=True)
dfls_df = dfls_df.iloc[:-3].reset_index(drop=True)

cfds_output_path = "data_cleaned_cfds_glencore.csv"
dfls_output_path = "data_cleaned_dfls_glencore.csv"
spreads_output_path = "data_cleaned_spreads_glencore.csv"
efp_output_path = "data_cleaned_efps_glencore.csv"

weekly_cfds_df.to_csv(cfds_output_path, index=False)
dfls_df.to_csv(dfls_output_path, index=False)
spreads_df.to_csv(spreads_output_path, index=False)

efps_from_dfls_df.to_csv("data_cleaned_efps_glencore.csv", index=False)
print("✅ Files saved:")
print(f" - CFDs: {cfds_output_path}")
print(f" - DFLs: {dfls_output_path}")
print(f" - Spreads: {spreads_output_path}")
print(f" - EFPs: {efp_output_path}")


✅ Files saved:
 - CFDs: data_cleaned_cfds_glencore.csv
 - DFLs: data_cleaned_dfls_glencore.csv
 - Spreads: data_cleaned_spreads_glencore.csv
 - EFPs: data_cleaned_efps_glencore.csv


In [ ]:
import pandas as pd

file_path = "marketclose 11062025.xls"
xls = pd.ExcelFile(file_path)
sheet_data = xls.parse(xls.sheet_names[0], header=None)

weekly_cfds_rows = []
cfds_section_found = False

for index, row in sheet_data.iterrows():
    row_str = str(row[0])

    if "CFD's" in row_str:
        cfds_section_found = True
        continue

    if cfds_section_found:
        if pd.notna(row[0]) and pd.notna(row[1]) and pd.notna(row[2]):
            weekly_cfds_rows.append([row[0], row[1], row[2]])
        elif pd.isna(row[0]) or "All Month" in str(row[0]):
            break

weekly_cfds_df = pd.DataFrame(weekly_cfds_rows, columns=['Week Range', 'Month', 'Value'])

dfls_rows = []
dfl_section_found = False

for index, row in sheet_data.iterrows():
    col_f = str(row[5]).strip() if pd.notna(row[5]) else ""
    col_g = row[6]

    if not dfl_section_found:
        if col_f == "Dated Front Line ICE":
            dfl_section_found = True
        continue

    if dfl_section_found:
        if "Calendar WTI/Brent Swap" in col_f:
            break

        if col_f and pd.notna(col_g):
            dfls_rows.append([col_f, col_g])

dfls_df = pd.DataFrame(dfls_rows, columns=['Month', 'Value'])

cfds_output_path = "data_cleaned_cfds_source.csv"
dfls_output_path = "data_cleaned_dfls_source.csv"

weekly_cfds_df.to_csv(cfds_output_path, index=False)
dfls_df.to_csv(dfls_output_path, index=False)

print("✅ Files saved:")
print(f" - CFDs: {cfds_output_path}")
print(f" - DFLs: {dfls_output_path}")

spreads_rows = []
spreads_started = False

for index, row in sheet_data.iterrows():
    col_k = str(row[10]).strip() if pd.notna(row[10]) else ""
    col_l = str(row[11]).strip() if pd.notna(row[11]) else ""
    col_m = row[12]

    if not spreads_started and "Brent Spreads" in col_l:
        spreads_started = True
        continue

    if spreads_started:
        if not col_k and not col_l and pd.isna(col_m):
            break
        if col_k and col_l and pd.notna(col_m):
            spreads_rows.append([col_k.strip(), col_l.strip(), col_m])

spreads_df = pd.DataFrame(spreads_rows, columns=["Month1", "Month2", "Value"])
spreads_df.to_csv("data_cleaned_spreads_source.csv", index=False)
print(" - Spreads: data_cleaned_spreads_source.csv")

brent_efps_rows = []
efp_section_started = False

for index, row in sheet_data.iterrows():
    col_a = str(row[0]).strip() if pd.notna(row[0]) else ""
    col_b = row[1]

    if not efp_section_started and "Brent EFP" in col_a:
        efp_section_started = True
        continue

    if efp_section_started:
        if not col_a and pd.isna(col_b):
            break
        if col_a and pd.notna(col_b):
            brent_efps_rows.append([col_a, col_b])

brent_efps_df = pd.DataFrame(brent_efps_rows, columns=["Month", "Value"])
brent_efps_df.to_csv("data_cleaned_brent_efps_source.csv", index=False)
print(" - Brent EFPs: data_cleaned_brent_efps_source.csv")


✅ Files saved:
 - CFDs: data_cleaned_cfds_source.csv
 - DFLs: data_cleaned_dfls_source.csv
 - Spreads: data_cleaned_spreads_source.csv
 - Brent EFPs: data_cleaned_brent_efps_source.csv


In [ ]:
import pandas as pd

df1 = pd.read_csv("/content/data_cleaned_cfds_glencore.csv")
df2 = pd.read_csv("/content/data_cleaned_cfds_source.csv")

df1.rename(columns={"Date Range": "Week Range"}, inplace=True)

df1["Week Range"] = df1["Week Range"].astype(str).str.strip()
df1["Month"] = df1["Month"].astype(str).str.strip()

df2["Week Range"] = df2["Week Range"].astype(str).str.strip()
df2["Month"] = df2["Month"].astype(str).str.strip()

merged = pd.merge(df1, df2, on=["Week Range", "Month"], how="outer", suffixes=('_file1', '_file2'))

for idx, row in merged.iterrows():
    val1 = row.get("Value_file1")
    val2 = row.get("Value_file2")

    if pd.isna(val1):
        print(f"❌ Missing in Glencore: {row['Week Range']} {row['Month']} → Value in Source = {val2}")

    elif pd.isna(val2):
        print(f"❌ Missing in Source: {row['Week Range']} {row['Month']} → Value in Glencore = {val1}")

    elif val1 != val2:
        print(f"❌ Mismatch on {row['Week Range']} {row['Month']} → Glencore: {val1}, Source: {val2}")


❌ Mismatch on 1-5/9 Nov → Glencore: 70.0, Source: 82


In [ ]:
import pandas as pd
import re

df1 = pd.read_csv("/content/data_cleaned_dfls_glencore.csv")
df2 = pd.read_csv("/content/data_cleaned_dfls_source.csv")

def normalize_month(month):
    if pd.isna(month):
        return month
    month = month.strip().lower()

    match = re.match(r'^20(\d{2})$', month)
    if match:
        return match.group(1)

    month = month.replace('cal', '')
    month = month.replace('-', '')
    return month

df1['Month_norm'] = df1['Month'].apply(normalize_month)
df2['Month_norm'] = df2['Month'].apply(normalize_month)

merged = pd.merge(df1, df2, on='Month_norm', how='outer', suffixes=('_file1', '_file2'))

for _, row in merged.iterrows():
    m1 = row.get("Month_file1")
    m2 = row.get("Month_file2")
    v1 = row.get("Value_file1")
    v2 = row.get("Value_file2")
    label = row["Month_norm"]

    if pd.isna(v1):
        print(f"❌ Missing in Glencore: {m2} → Value in Source = {v2}")
    elif pd.isna(v2):
        print(f"❌ Missing in Source: {m1} → Value in Glencore = {v1}")
    elif float(v1) != float(v2):
        print(f"❌ Mismatch on '{label}' → Glencore: {v1}, Source: {v2}")


❌ Missing in Source: Dec → Value in Glencore = 7.0
❌ Missing in Source: Feb → Value in Glencore = 6.0
❌ Missing in Source: Jan → Value in Glencore = 7.0
❌ Missing in Source: Nov → Value in Glencore = 19.0
❌ Missing in Source: Oct → Value in Glencore = 39.0
❌ Missing in Source: Sep → Value in Glencore = 61.0


In [ ]:
import pandas as pd

df1 = pd.read_csv("/content/data_cleaned_efps_glencore.csv")
df2 = pd.read_csv("/content/data_cleaned_brent_efps_source.csv")

merged = pd.merge(df1, df2, on='Month', how='outer', suffixes=('_data1', '_data2'))

merged['Value_data1'] = pd.to_numeric(merged['Value_data1'], errors='coerce')
merged['Value_data2'] = pd.to_numeric(merged['Value_data2'], errors='coerce')

for _, row in merged.iterrows():
    month = row['Month']
    v1 = row['Value_data1']
    v2 = row['Value_data2']

    if pd.isna(v1):
        print(f"Missing value in glencore for {month}, source value is {v2}")
    elif pd.isna(v2):
        print(f"Missing value in source for {month}, glencore value is {v1}")
    elif v1 != v2:
        print(f"Value in glencore is {v1}, value in source is {v2} — mismatch")


Missing value in glencore for Nov, source value is 7


In [ ]:
import pandas as pd

df1 = pd.read_csv("data_cleaned_spreads_glencore.csv")
df2 = pd.read_csv("data_cleaned_spreads_source.csv")

df2['Spread'] = df2['Month1'] + "/" + df2['Month2'] + " ICE"

merged = pd.merge(df2, df1, on='Spread', how='outer', suffixes=('_data2', '_data1'))

merged['Value_data1'] = pd.to_numeric(merged['Value_data1'], errors='coerce')
merged['Value_data2'] = pd.to_numeric(merged['Value_data2'], errors='coerce')

for _, row in merged.iterrows():
    spread = row['Spread']
    v1 = row['Value_data1']
    v2 = row['Value_data2']

    if pd.isna(v1):
        print(f"Missing value in glencore for {spread}, source value is {v2}")
    elif pd.isna(v2):
        print(f"Missing value in source for {spread}, glencore value is {v1}")
    elif v1 != v2:
        print(f"Value in glencore is {v1}, value in source is {v2} — mismatch for {spread}")


Missing value in glencore for Dec/Jan ICE, source value is 14.0
Missing value in source for Dec/Mar26, glencore value is 22.0
Missing value in source for Dec25/Dec26, glencore value is 17.0
Missing value in source for Dec26/Dec27, glencore value is -33.0
Missing value in glencore for Jan/Feb ICE, source value is 5.0
Missing value in source for Sep/Dec25, glencore value is 151.0


In [ ]:
import pandas as pd

def load_excel_sheet(file_path: str, sheet_name: str = "Sheet1") -> pd.DataFrame:
    """
    Loads the specified Excel sheet into a pandas DataFrame without headers.

    Args:
        file_path (str): Path to the Excel file.
        sheet_name (str): Name of the sheet to parse.

    Returns:
        pd.DataFrame: Raw data from the specified sheet.
    """
    xls = pd.ExcelFile(file_path)
    return xls.parse(sheet_name, header=None)

def extract_weekly_cfds(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts weekly CFD data starting from 'Week1' until the first non-'Week' row.

    It captures columns: Week name, Date Range, Month, and Value.

    Args:
        sheet_data (pd.DataFrame): Raw Excel sheet data.

    Returns:
        pd.DataFrame: Cleaned DataFrame containing weekly CFD data.
    """
    weekly_cfds_rows = []
    collecting = False

    for index, row in sheet_data.iterrows():
        first_cell = str(row[0]).strip()

        if first_cell.startswith("Week1"):
            collecting = True

        if collecting:
            if str(first_cell).startswith("Week"):
                weekly_cfds_rows.append([row[0], row[1], row[3], row[4]])
            else:
                break

    return pd.DataFrame(weekly_cfds_rows, columns=['Week', 'Date Range', 'Month', 'Value']).reset_index(drop=True)

def extract_dfls(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts DFL data by collecting non-null values from columns 7 and 8.

    Args:
        sheet_data (pd.DataFrame): Raw Excel sheet data.

    Returns:
        pd.DataFrame: DataFrame containing DFL data with 'Month' and 'Value' columns.
    """
    dfls_rows = []

    for index, row in sheet_data.iterrows():
        month = row[7]
        value = row[8]

        if pd.notna(month) and pd.notna(value):
            dfls_rows.append([month, value])

    return pd.DataFrame(dfls_rows, columns=['Month', 'Value'])

def extract_spreads(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts spread data by looking for rows where column 3 contains "ICE" or "/"
    and column 4 has a value.

    Args:
        sheet_data (pd.DataFrame): Raw Excel sheet data.

    Returns:
        pd.DataFrame: DataFrame containing spread labels and their values.
    """
    spreads_rows = []

    for index, row in sheet_data.iterrows():
        col3 = row[3]
        col4 = row[4]

        if pd.notna(col3) and pd.notna(col4):
            if "ICE" in str(col3) or "/" in str(col3):
                spreads_rows.append([col3, col4])

    return pd.DataFrame(spreads_rows, columns=['Spread', 'Value'])

def separate_efps(dfls_df: pd.DataFrame) -> (pd.DataFrame, pd.DataFrame):
    """
    Separates the last three rows from DFLs as EFPs and returns both DataFrames.

    Args:
        dfls_df (pd.DataFrame): Complete DFL DataFrame.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: DFL data excluding EFPs, and EFP data.
    """
    efps_df = dfls_df.tail(3).reset_index(drop=True)
    dfls_df = dfls_df.iloc[:-3].reset_index(drop=True)
    return dfls_df, efps_df

def save_to_csv(df: pd.DataFrame, output_path: str):
    """
    Saves the given DataFrame to a CSV file.

    Args:
        df (pd.DataFrame): Data to save.
        output_path (str): Path for the output CSV file.
    """
    df.to_csv(output_path, index=False)

def main():
    """
    Main execution function that processes the Excel file:
    - Extracts weekly CFDs, DFLs, Spreads, and EFPs
    - Saves each dataset to separate CSV files
    - Prints output confirmation
    """
    file_path = "/content/Glencore recap 11062025.xlsx"

    sheet_data = load_excel_sheet(file_path)

    weekly_cfds_df = extract_weekly_cfds(sheet_data)
    dfls_df = extract_dfls(sheet_data)
    spreads_df = extract_spreads(sheet_data)
    dfls_df, efps_df = separate_efps(dfls_df)

    cfds_output_path = "data_cleaned_cfds_glencore.csv"
    dfls_output_path = "data_cleaned_dfls_glencore.csv"
    spreads_output_path = "data_cleaned_spreads_glencore.csv"
    efp_output_path = "data_cleaned_efps_glencore.csv"

    save_to_csv(weekly_cfds_df, cfds_output_path)
    save_to_csv(dfls_df, dfls_output_path)
    save_to_csv(spreads_df, spreads_output_path)
    save_to_csv(efps_df, efp_output_path)

    print("✅ Files saved:")
    print(f" - CFDs: {cfds_output_path}")
    print(f" - DFLs: {dfls_output_path}")
    print(f" - Spreads: {spreads_output_path}")
    print(f" - EFPs: {efp_output_path}")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

def load_excel_sheet(file_path: str) -> pd.DataFrame:
    """
    Loads the first sheet of an Excel file as a pandas DataFrame with no headers.

    Args:
        file_path (str): Path to the Excel file.

    Returns:
        pd.DataFrame: DataFrame containing all sheet data.
    """
    xls = pd.ExcelFile(file_path)
    return xls.parse(xls.sheet_names[0], header=None)

def extract_weekly_cfds(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts CFD data starting after the row containing "CFD's".
    Stops when it encounters an empty row or a row containing "All Month".

    Args:
        sheet_data (pd.DataFrame): The entire Excel sheet as a DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing columns ['Week Range', 'Month', 'Value'].
    """
    weekly_cfds_rows = []
    cfds_section_found = False

    for _, row in sheet_data.iterrows():
        row_str = str(row[0])

        if "CFD's" in row_str:
            cfds_section_found = True
            continue

        if cfds_section_found:
            if pd.notna(row[0]) and pd.notna(row[1]) and pd.notna(row[2]):
                weekly_cfds_rows.append([row[0], row[1], row[2]])
            elif pd.isna(row[0]) or "All Month" in str(row[0]):
                break

    return pd.DataFrame(weekly_cfds_rows, columns=['Week Range', 'Month', 'Value'])

def extract_dfls(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts DFL data from the sheet after finding the row "Dated Front Line ICE".
    Stops before the row containing "Calendar WTI/Brent Swap".

    Args:
        sheet_data (pd.DataFrame): The entire Excel sheet as a DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing DFL data with ['Month', 'Value'].
    """
    dfls_rows = []
    dfl_section_found = False

    for _, row in sheet_data.iterrows():
        col_f = str(row[5]).strip() if pd.notna(row[5]) else ""
        col_g = row[6]

        if not dfl_section_found:
            if col_f == "Dated Front Line ICE":
                dfl_section_found = True
            continue

        if dfl_section_found:
            if "Calendar WTI/Brent Swap" in col_f:
                break
            if col_f and pd.notna(col_g):
                dfls_rows.append([col_f, col_g])

    return pd.DataFrame(dfls_rows, columns=['Month', 'Value'])

def extract_spreads(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts Brent spread data after identifying the row containing "Brent Spreads".
    Collects rows with three non-null values and stops at a fully empty row.

    Args:
        sheet_data (pd.DataFrame): The entire Excel sheet as a DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing ['Month1', 'Month2', 'Value'].
    """
    spreads_rows = []
    spreads_started = False

    for _, row in sheet_data.iterrows():
        col_k = str(row[10]).strip() if pd.notna(row[10]) else ""
        col_l = str(row[11]).strip() if pd.notna(row[11]) else ""
        col_m = row[12]

        if not spreads_started and "Brent Spreads" in col_l:
            spreads_started = True
            continue

        if spreads_started:
            if not col_k and not col_l and pd.isna(col_m):
                break
            if col_k and col_l and pd.notna(col_m):
                spreads_rows.append([col_k, col_l, col_m])

    return pd.DataFrame(spreads_rows, columns=["Month1", "Month2", "Value"])

def extract_brent_efps(sheet_data: pd.DataFrame) -> pd.DataFrame:
    """
    Extracts Brent EFP data starting after the row containing "Brent EFP".
    Stops at the first fully empty row and collects pairs of [Month, Value].

    Args:
        sheet_data (pd.DataFrame): The entire Excel sheet as a DataFrame.

    Returns:
        pd.DataFrame: DataFrame containing Brent EFPs with ['Month', 'Value'].
    """
    brent_efps_rows = []
    efp_section_started = False

    for _, row in sheet_data.iterrows():
        col_a = str(row[0]).strip() if pd.notna(row[0]) else ""
        col_b = row[1]

        if not efp_section_started and "Brent EFP" in col_a:
            efp_section_started = True
            continue

        if efp_section_started:
            if not col_a and pd.isna(col_b):
                break
            if col_a and pd.notna(col_b):
                brent_efps_rows.append([col_a, col_b])

    return pd.DataFrame(brent_efps_rows, columns=["Month", "Value"])

def save_to_csv(df: pd.DataFrame, output_path: str):
    """
    Saves a DataFrame to a CSV file without the index column.

    Args:
        df (pd.DataFrame): The data to save.
        output_path (str): Output path of the CSV file.
    """
    df.to_csv(output_path, index=False)

def main():
    """
    Main function to extract CFDs, DFLs, Spreads, and Brent EFPs from an Excel file,
    and save each section into separate CSV files. Also prints the output paths.
    """
    file_path = "marketclose 11062025.xls"
    sheet_data = load_excel_sheet(file_path)

    weekly_cfds_df = extract_weekly_cfds(sheet_data)
    dfls_df = extract_dfls(sheet_data)
    spreads_df = extract_spreads(sheet_data)
    brent_efps_df = extract_brent_efps(sheet_data)

    save_to_csv(weekly_cfds_df, "data_cleaned_cfds_source.csv")
    save_to_csv(dfls_df, "data_cleaned_dfls_source.csv")
    save_to_csv(spreads_df, "data_cleaned_spreads_source.csv")
    save_to_csv(brent_efps_df, "data_cleaned_brent_efps_source.csv")

    print("✅ Files saved:")
    print(" - CFDs: data_cleaned_cfds_source.csv")
    print(" - DFLs: data_cleaned_dfls_source.csv")
    print(" - Spreads: data_cleaned_spreads_source.csv")
    print(" - Brent EFPs: data_cleaned_brent_efps_source.csv")

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

def load_and_prepare_csv(path: str, rename_cols: dict = None) -> pd.DataFrame:
    """
    Loads a CSV and optionally renames columns, then strips whitespace from 'Week Range' and 'Month'.

    Args:
        path (str): Path to the CSV file.
        rename_cols (dict, optional): Dictionary for renaming columns.

    Returns:
        pd.DataFrame: Cleaned DataFrame with normalized keys.
    """
    df = pd.read_csv(path)
    if rename_cols:
        df.rename(columns=rename_cols, inplace=True)

    df["Week Range"] = df["Week Range"].astype(str).str.strip()
    df["Month"] = df["Month"].astype(str).str.strip()

    return df

def compare_cfd_values(df1: pd.DataFrame, df2: pd.DataFrame):
    """
    Compares the 'Value' fields between two CFD DataFrames based on 'Week Range' and 'Month'.
    Prints differences, including mismatches and missing values from either file.

    Args:
        df1 (pd.DataFrame): First CFD DataFrame (e.g., Glencore).
        df2 (pd.DataFrame): Second CFD DataFrame (e.g., Source).
    """
    merged = pd.merge(df1, df2, on=["Week Range", "Month"], how="outer", suffixes=('_file1', '_file2'))

    for _, row in merged.iterrows():
        val1 = row.get("Value_file1")
        val2 = row.get("Value_file2")

        if pd.isna(val1):
            print(f"❌ Missing in Glencore: {row['Week Range']} {row['Month']} → Value in Source = {val2}")

        elif pd.isna(val2):
            print(f"❌ Missing in Source: {row['Week Range']} {row['Month']} → Value in Glencore = {val1}")

        elif val1 != val2:
            print(f"❌ Mismatch on {row['Week Range']} {row['Month']} → Glencore: {val1}, Source: {val2}")

def main():
    """
    Main function to compare CFD values between Glencore and Source datasets.
    Identifies mismatched or missing values between the two.
    """
    path_glencore = "/content/data_cleaned_cfds_glencore.csv"
    path_source = "/content/data_cleaned_cfds_source.csv"

    df_glencore = load_and_prepare_csv(path_glencore, rename_cols={"Date Range": "Week Range"})
    df_source = load_and_prepare_csv(path_source)

    compare_cfd_values(df_glencore, df_source)

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd
import re

def normalize_month(month: str) -> str:
    """
    Normalizes month strings to a comparable format by:
    - Converting to lowercase
    - Removing 'cal' and hyphens
    - Stripping whitespace
    - Converting '2026' → '26', etc.

    Args:
        month (str): Original month string.

    Returns:
        str: Normalized month string, or original if NaN.
    """
    if pd.isna(month):
        return month
    month = month.strip().lower()

    match = re.match(r'^20(\d{2})$', month)
    if match:
        return match.group(1)

    month = month.replace('cal', '')
    month = month.replace('-', '')
    return month

def load_and_prepare_dfl(path: str) -> pd.DataFrame:
    """
    Loads a DFL CSV and applies month normalization.

    Args:
        path (str): Path to the DFL CSV file.

    Returns:
        pd.DataFrame: DataFrame with normalized month column 'Month_norm'.
    """
    df = pd.read_csv(path)
    df['Month_norm'] = df['Month'].apply(normalize_month)
    return df

def compare_dfl_values(df1: pd.DataFrame, df2: pd.DataFrame):
    """
    Merges two DFL DataFrames on normalized months and prints mismatches,
    missing entries, and inconsistent values.

    Args:
        df1 (pd.DataFrame): Glencore DFL data.
        df2 (pd.DataFrame): Source DFL data.
    """
    merged = pd.merge(df1, df2, on='Month_norm', how='outer', suffixes=('_file1', '_file2'))

    for _, row in merged.iterrows():
        m1 = row.get("Month_file1")
        m2 = row.get("Month_file2")
        v1 = row.get("Value_file1")
        v2 = row.get("Value_file2")
        label = row["Month_norm"]

        if pd.isna(v1):
            print(f"❌ Missing in Glencore: {m2} → Value in Source = {v2}")
        elif pd.isna(v2):
            print(f"❌ Missing in Source: {m1} → Value in Glencore = {v1}")
        elif float(v1) != float(v2):
            print(f"❌ Mismatch on '{label}' → Glencore: {v1}, Source: {v2}")

def main():
    """
    Main function to:
    - Load Glencore and Source DFL data
    - Normalize months
    - Compare and report mismatches or missing values
    """
    df1 = load_and_prepare_dfl("/content/data_cleaned_dfls_glencore.csv")
    df2 = load_and_prepare_dfl("/content/data_cleaned_dfls_source.csv")
    compare_dfl_values(df1, df2)

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

def load_and_clean_efp(path: str, value_col: str = "Value") -> pd.DataFrame:
    """
    Loads an EFP CSV and ensures the value column is numeric.

    Args:
        path (str): Path to the EFP CSV file.
        value_col (str): Column name that holds the numeric value.

    Returns:
        pd.DataFrame: Cleaned EFP DataFrame with numeric values.
    """
    df = pd.read_csv(path)
    df[value_col] = pd.to_numeric(df[value_col], errors='coerce')
    return df

def compare_efp_values(df1: pd.DataFrame, df2: pd.DataFrame):
    """
    Compares EFP values from two sources by merging on the 'Month' column.
    Prints out mismatches and missing values.

    Args:
        df1 (pd.DataFrame): Glencore EFP data.
        df2 (pd.DataFrame): Source Brent EFP data.
    """
    merged = pd.merge(df1, df2, on='Month', how='outer', suffixes=('_data1', '_data2'))

    for _, row in merged.iterrows():
        month = row['Month']
        v1 = row['Value_data1']
        v2 = row['Value_data2']

        if pd.isna(v1):
            print(f"❌ Missing value in Glencore for {month}, Source value = {v2}")
        elif pd.isna(v2):
            print(f"❌ Missing value in Source for {month}, Glencore value = {v1}")
        elif v1 != v2:
            print(f"❌ Mismatch in {month} → Glencore: {v1}, Source: {v2}")

def main():
    """
    Loads and compares EFP values from Glencore and Source files,
    identifying any mismatches or missing data.
    """
    df1 = load_and_clean_efp("/content/data_cleaned_efps_glencore.csv")
    df2 = load_and_clean_efp("/content/data_cleaned_brent_efps_source.csv")
    compare_efp_values(df1, df2)

if __name__ == "__main__":
    main()


In [ ]:
import pandas as pd

def load_and_prepare_spreads(glencore_path: str, source_path: str) -> (pd.DataFrame, pd.DataFrame):
    """
    Loads and prepares spread data from Glencore and Source files.
    For the source file, constructs a unified 'Spread' column from Month1 and Month2.

    Args:
        glencore_path (str): Path to Glencore spreads CSV.
        source_path (str): Path to Source spreads CSV.

    Returns:
        Tuple[pd.DataFrame, pd.DataFrame]: Prepared Glencore and Source DataFrames.
    """
    df1 = pd.read_csv(glencore_path)
    df2 = pd.read_csv(source_path)

    df2['Spread'] = df2['Month1'].astype(str).str.strip() + "/" + df2['Month2'].astype(str).str.strip() + " ICE"

    df1['Value'] = pd.to_numeric(df1['Value'], errors='coerce')
    df2['Value'] = pd.to_numeric(df2['Value'], errors='coerce')

    return df1, df2

def compare_spread_values(df_glencore: pd.DataFrame, df_source: pd.DataFrame):
    """
    Merges and compares spread values from Glencore and Source on the 'Spread' column.
    Prints mismatches and missing values from either side.

    Args:
        df_glencore (pd.DataFrame): Glencore spread data.
        df_source (pd.DataFrame): Source spread data with constructed 'Spread' column.
    """
    merged = pd.merge(df_source, df_glencore, on='Spread', how='outer', suffixes=('_data2', '_data1'))

    for _, row in merged.iterrows():
        spread = row['Spread']
        v1 = row['Value_data1']
        v2 = row['Value_data2']

        if pd.isna(v1):
            print(f"❌ Missing in Glencore for {spread}, Source value = {v2}")
        elif pd.isna(v2):
            print(f"❌ Missing in Source for {spread}, Glencore value = {v1}")
        elif v1 != v2:
            print(f"❌ Mismatch in {spread} → Glencore: {v1}, Source: {v2}")

def main():
    """
    Main execution function for comparing Brent Spread values between
    Glencore and Source datasets. Outputs mismatches and missing data.
    """
    glencore_path = "data_cleaned_spreads_glencore.csv"
    source_path = "data_cleaned_spreads_source.csv"

    df_glencore, df_source = load_and_prepare_spreads(glencore_path, source_path)
    compare_spread_values(df_glencore, df_source)

if __name__ == "__main__":
    main()
